# 1.0 Libraries

In [1]:
!pip install langid
!pip install gensim
#!pip install -U spacy
!python -m spacy download it_core_news_sm
!pip install langdetect


Traceback (most recent call last):
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\runpy.py", line 188, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\runpy.py", line 147, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packages\spacy\__init__.py", line 13, in <module>
    from . import pipeline  # noqa: F401
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packages\spacy\pipeline\__init__.py", line 1, in <module>
    from .attributeruler import AttributeRuler
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packages\spacy\pipeline\attributeruler.py", line 8, in <module>
    from ..language import Language
  File "c:\Users\35193\miniconda3\envs\cuda_env\lib\site-packag

#Imports

In [2]:
import numpy as np
import pandas as pd

# from google.colab import drive
# drive.mount('/content/gdrive')

import re
import json
import gensim.downloader as api

import langid
from gensim.models import Word2Vec

# from sense2vec import Sense2Vec
import random

import torch
"""from transformers import RobertaTokenizer
from transformers import RobertaForMultipleChoice
from torch.distributions import Categorical
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline"""

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# import seaborn as sns
import matplotlib.pyplot as plt

# import nltk
# nltk.download('omw-1.4')
# nltk.download('wordnet')
# from nltk.corpus import wordnet as wn

import time
import warnings
warnings.filterwarnings('ignore')

# 2.0 Set up

In [3]:
import random

In [4]:
def load_data(data_path, gold_path):
  count = 0
  hypernyms_dict = {}
  with open(data_path, "r", encoding = 'utf-8') as data_file, open(gold_path, "r", encoding = 'utf-8') as gold_file:
    for data_line, gold_line in zip(data_file, gold_file):
      term_list = [term for term in data_line.split()[:-1]]
      term = " ".join(term_list)
      hypernyms = [hypernym.replace("\n", "") for hypernym in gold_line.split("\t")]
      hypernyms_dict[term] = hypernyms
      count += 1

      if count == 20:
        break

  return hypernyms_dict

In [5]:
#  PARTIAL italian training data
train_hypernyms = load_data("1B.italian.training.data.txt", "1B.italian.training.gold.txt")

# PARTIAL italian test data
test_hypernyms = load_data("1B.italian.test.data.txt", "1B.italian.test.gold.txt")

In [6]:
print(train_hypernyms)

{'sesto': ['grado', 'numero ordinale', 'frazione', 'carica'], 'Sigillo': ['comune', 'municipalità', 'comune italiano', 'frazione', 'paese', 'quartiere', 'comunità', 'borgo'], 'toga': ['comune', 'mantella', 'indumento', 'vestito', 'mantello', 'veste', 'abito', 'costume'], 'Eduard Buchner': ['accademico', 'persona', 'professore', 'biochimico', 'scienziato', 'chimico', 'scopritore', 'docente'], 'montagna': ['panorama', 'grande quantità', 'rilievo'], 'Renato Guttuso': ['pittore', 'persona', 'artista', 'politico'], 'cardamomo': ['pianta', 'aroma', 'tracheobionta', 'vegetale', 'pianta da fiore', 'magnoliophyta'], 'musical': ['rappresentazione', 'spettacolo', 'spettacolo teatrale', 'show'], 'vicino': ['persona'], 'rinnovo': ['sostituzione', 'prosieguo', 'proroga', 'proseguimento', 'replica', 'continuamento', 'prosecuzione'], 'rampollo': ['parente', 'congiunto', 'persona', 'famigliare'], 'piccione': ['uccello', 'animale', 'volatile', 'vertebrato'], 'cernita': ['comune'], 'gettone': ['dischetto

#Generate Distractors from italian dataset


In [7]:
!git clone https://github.com/napolux/paroleitaliane.git
%cd /content/Parole_Italiane

[WinError 3] O sistema não conseguiu localizar o caminho especificado: '/content/Parole_Italiane'
c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_26


Cloning into 'paroleitaliane'...


In [8]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
!gunzip cc.it.300.bin.gz
!pip install fasttext

'wget' is not recognized as an internal or external command,
operable program or batch file.


'gunzip' is not recognized as an internal or external command,
operable program or batch file.


  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for fasttext
Failed to build fasttext


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
      C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\fasttext
      copying python\fasttext_module\fasttext\FastText.py -> build\lib.win-amd64-cpython-310\fasttext
      copying python\fasttext_module\fasttext\__init__.py -> build\lib.win-amd64-cpython-310\fasttext
      creating build\lib.win-amd64-cpython-310\fasttext\util
      copying python\fasttext_module\fasttext\util\util.py -> build\lib.win

In [11]:
with open('280000_parole_italiane.txt', 'r') as file:
  italian_word_list=[]
  for line in file:
    italian_word_list.append(line)


In [12]:
import fasttext
import numpy as np
import fasttext.util

def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

def check_letter(word1, word2):

    common_letters = set(word1).intersection(set(word2))

    threshold = len(word1) * 0.4

    if len(common_letters) >= threshold:
        return False
    else:
        return True

In [15]:
def find_distractors(italian_word_list ,target_word, number, model):
  target_vector = model.get_word_vector(target_word)

  word_list = italian_word_list
  similar_words = []
  th = 0.5

  while(len(similar_words)< number):
    similar_words = []
    for word in word_list:
        word_vector = model.get_word_vector(word)
        similarity = cosine_similarity(target_vector, word_vector)


        if similarity > th and check_letter(target_word, word):
            similar_words.append(word.replace('\n', ''))
    th = th - 0.025
  results = []
  while(len(results)<3):
    index = random.randint(0, len(similar_words)-1)
    if similar_words[index] not in(results):
      results.append(similar_words[index])

  return results


ft = fasttext.load_model('cc.it.300.bin')

print(find_distractors(italian_word_list, "uccello", 3 ,ft))


ValueError: cc.it.300.bin has wrong file format!

# 4.0 Create entries

In [ ]:
def save_jsonl(file_path, data, model, italian_word_list):
  id_seq = 0
  with open(file_path, "w") as output_file:
    for term, hypernyms in data.items():
      for hypernym in hypernyms:
        distractors = find_distractors(italian_word_list, hypernym, 3, model)
        entries = (hypernym, *distractors)
        choices = list(entries)
        random.shuffle(choices) # to create randomness
        reformatted_json_data = {
              'id' : id_seq,
              'text': term,
              'choices': choices,
              'label' : choices.index(hypernym)
        }
        json.dump(reformatted_json_data, output_file)
        output_file.write("\n")
        id_seq +=1

In [ ]:
def read_lines_jsonl(file_path, num_lines):
  with open(file_path, 'r') as f:
    json_list = list(f)
    for line in json_list[:num_lines]:
      data = json.loads(line)
      print(data)

In [ ]:
save_jsonl("hypernym_discovery-task26-train-data.jsonl", train_hypernyms, ft, italian_word_list)
read_lines_jsonl("hypernym_discovery-task26-train-data.jsonl", num_lines = 10) 

KeyboardInterrupt: 

In [ ]:
# test jsonl file
save_jsonl("hypernym_discovery-task26-test-data.jsonl", train_hypernyms, ft, italian_word_list)
read_lines_jsonl("hypernym_discovery-task26-test-data.jsonl", num_lines = 10) 

KeyboardInterrupt: 